In [1]:
import os
from openai import OpenAI

In [2]:
import datetime
import json

In [3]:
# KEY NEEDS TO BE DEFINED AS AN ENV
# OR DEFINE IT HERE
# OPENAI_API_KEY = "KEY HERE"
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
client_openai = OpenAI()

In [5]:
from httpx import ConnectTimeout, Timeout
NUM_TRIES = 3
TIMEOUT = 20


def llm_wrapper(
    model_name, 
    messages,
    response_format=None
):

    n = 1
    while n<NUM_TRIES:
        try:

            # messages = [
            #     {'role': 'system', 'content': system_prompt},
            #     {'role': 'user', 'content': user_message},
            # ]
            response = client_openai.beta.chat.completions.parse(
                model=model_name,
                temperature=1.,
                top_p=1.,
                messages=messages,
                response_format=response_format
            )    

            if response_format is None:
                return response.choices[0].message.content #, prompt_tokens, completion_tokens
            else:
                return response.choices[0].message.parsed.json()
        except ConnectTimeout:
            n += 1
    
    raise Exception(f"Couldn't get response from OpenAI in {NUM_TRIES} trials!")

In [6]:
CUSTOMER_PROMPT_BASE = """You are a Customer who will converse with a Sales Rep.
You are looking to buy the product {product_name}.

FOLLOW THESE INSTRUCTIONS DURING THE CONVERSATION:
- Initially just Greet the sales rep.
- Then state your intent and gradually ask questions to make a decision to buy.
- After you think you get the adequate information you can proceed to buy the product or not.
- YOU ARE THE CUSTOMER NOT THE SALES REP.
"""

SALES_REP_PROMPT_BASE = """You are a Sales Rep helping customers about their questions.

FOLLOW THESE INSTRUCTIONS DURING THE CONVERSATION:
- You are helpful.
- You simulate spoken conversation. Do not generate structured text.
- YOU ARE THE SALES REP NOT THE CUSTOMER.
"""


In [7]:
exit_tool = [
  {
      "type": "function",
      "function": {
          "name": "buy_decision",
          "description": "Call this function when you think you have gained all information you need from the conversation and would like to buy or not.",
          "parameters": {
              "type": "object",
              "properties": {
                 "decision": {
                    "enum": ["buy", "not_buy"],
                    "type": "string",
                    "description": "you decision about the product",
                },
              },
              "required": ["decision"],
              "additionalProperties": False,
          },
      }
  }
]

In [8]:
def message_history_converter(message_history):
    converted_history = []
    for m in message_history:
        if m["role"] != "system":
            converted_history.append(
                {
                    "role": "user" if m["role"] == "assistant" else "assistant",
                    "content": m["content"],
                }
            )
    return converted_history

In [9]:
model_name = 'gpt-4o-mini'
temperature = 1.0
top_p=1.0

In [10]:
# Prompt customization
PRODUCT_NAME = "iphone 14"
CUSTOMER_PROMPT = CUSTOMER_PROMPT_BASE.format(product_name=PRODUCT_NAME)
SALES_REP_PROMPT = SALES_REP_PROMPT_BASE

In [14]:
MAX_NUM_ROUNDS = 10
tools = None
tool_choice = None
wrap_up_after = 4
message_history = []
num_rounds = 0

while num_rounds < MAX_NUM_ROUNDS:
    print(f'Round: {num_rounds}')
    # we give customer a chance to early exit 
    # after some turns
    if (num_rounds >= wrap_up_after):
        tools = exit_tool
        tool_choice = "auto"
    
    customer_message_history = [
        {"role": "system", "content": CUSTOMER_PROMPT}
    ] + message_history_converter(message_history)
    
    #print(tool_choice)
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=temperature,
        top_p=top_p,
        messages=customer_message_history,
        tools=tools,
        tool_choice=tool_choice,
    )
    customer_message = response.choices[0].message
    
    if customer_message.tool_calls is None:
        message_history.append({"role": "user", "content": customer_message.content})
    else:
        #print('tool call')
        if customer_message.content is not None:
            message_history.append(
                {"role": "user", "content": customer_message.content}
            )
        break

    sales_rep_message_history = [
        {"role": "system", "content": SALES_REP_PROMPT}
    ] + message_history
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=temperature,
        top_p=top_p,
        messages=sales_rep_message_history,
    )
    sales_rep_message = response.choices[0].message
    message_history.append({"role": "assistant", "content": sales_rep_message.content})
    
    num_rounds += 1

Round: 0
Round: 1
Round: 2
Round: 3
Round: 4
Round: 5
Round: 6
Round: 7
Round: 8
Round: 9


In [15]:
message_history

[{'role': 'user', 'content': 'Hello!'},
 {'role': 'assistant', 'content': 'Hi there! How can I assist you today?'},
 {'role': 'user',
  'content': "I'm interested in buying the iPhone 14. Can you provide me with some information about it?"},
 {'role': 'assistant',
  'content': "Absolutely! The iPhone 14 features a stunning Super Retina XDR display, great camera capabilities, and impressive performance with the A15 Bionic chip. It comes in different colors and storage options. Are there any specific features you're particularly interested in, like the camera or battery life?"},
 {'role': 'user',
  'content': "I'm definitely interested in the camera features. What can you tell me about it?"},
 {'role': 'assistant',
  'content': "The camera on the iPhone 14 is really impressive! It has a dual-camera system with a 12MP wide and 12MP ultra-wide lens. The low-light performance is enhanced thanks to the larger sensor and improved image stabilization, so you’ll get great shots even in dim cond

In [13]:
fname = f"conversation_{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.json"
with open(fname, 'w') as ofile:
    json.dump(message_history,ofile)